In [1]:
import torch
import torchvision
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# === Feature flags ===
use_dropout            = True
use_leaky_relu         = True
use_grad_clipped_relu  = True   # applies gradient clipping if True
use_batchnorm          = True

# === Hyperparams ===
batch_size = 4
lr         = 1e-3
momentum   = 0.9
num_epochs = 20
grad_clip_max_norm = 1.0

# === Device ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# === Data transforms ===
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,)*3, (0.5,)*3)
])

# === Datasets & loaders ===
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,  download=True, transform=transform)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True,  num_workers=2)
testloader  = torch.utils.data.DataLoader(testset,  batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

# === Model definition ===
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # conv layers
        self.conv1 = nn.Conv2d(3,   32, 5)
        self.conv2 = nn.Conv2d(32,  64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        self.pool  = nn.MaxPool2d(2,2)

        # optional batchnorm
        if use_batchnorm:
            self.bn1 = nn.BatchNorm2d(32)
            self.bn2 = nn.BatchNorm2d(64)
            self.bn3 = nn.BatchNorm2d(128)

        # fully connected
        self.fc1 = nn.Linear(128*1*1, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        if use_batchnorm:
            self.bn4 = nn.BatchNorm1d(120)
            self.bn5 = nn.BatchNorm1d(84)

        # dropout
        if use_dropout:
            # self.drop = nn.Dropout(p=0.5)
            self.drop=nn.Dropout(p=0.2)

        # optional leaky ReLU
        if use_leaky_relu:
            self.lrelu = nn.LeakyReLU(negative_slope=0.01, inplace=True)

    def forward(self, x):
        act = self.lrelu if use_leaky_relu else F.relu

        x = self.conv1(x)
        if use_batchnorm: x = self.bn1(x)
        x = act(x)
        x = self.pool(x)

        x = self.conv2(x)
        if use_batchnorm: x = self.bn2(x)
        x = act(x)
        x = self.pool(x)

        x = self.conv3(x)
        if use_batchnorm: x = self.bn3(x)
        x = act(x)

        x = x.view(-1, 128*1*1)

        x = self.fc1(x)
        if use_batchnorm: x = self.bn4(x)
        x = act(x)
        if use_dropout:   x = self.drop(x)

        x = self.fc2(x)
        if use_batchnorm: x = self.bn5(x)
        x = act(x)
        if use_dropout:   x = self.drop(x)

        x = self.fc3(x)
        return x

net = Net().to(device)

# === Loss & optimizer ===
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(net.parameters(), lr=lr)

# === Training loop ===
for epoch in range(1, num_epochs+1):
    net.train()
    running_loss = 0.0
    correct_train = 0
    total_train   = 0

    for i, (inputs, labels) in enumerate(trainloader, 1):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        # print(inputs.shape)
        outputs = net(inputs)
        # print(outputs.shape)
        loss = criterion(outputs, labels)
        loss.backward()

        if use_grad_clipped_relu:
            torch.nn.utils.clip_grad_norm_(net.parameters(), grad_clip_max_norm)

        optimizer.step()

        # accumulate loss
        running_loss += loss.item()
        # training accuracy
        _, preds = outputs.max(1)
        correct_train += (preds == labels).sum().item()
        total_train   += labels.size(0)

        if i % 2000 == 0:
            avg_loss = running_loss / 2000
            print(f"[Epoch {epoch}, Batch {i}]  loss: {avg_loss:.3f}")
            running_loss = 0.0

    train_acc = 100 * correct_train / total_train
    print(f"Epoch {epoch}  ➜  Training Accuracy: {train_acc:.2f}%")

    # --- evaluate on test set each epoch ---
    net.eval()
    correct_test = 0
    total_test   = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, preds = outputs.max(1)
            correct_test += (preds == labels).sum().item()
            total_test   += labels.size(0)
    test_acc = 100 * correct_test / total_test
    print(f"Epoch {epoch}  ➜  Test    Accuracy: {test_acc:.2f}%\n")

print("Finished Training\n")

# === Final evaluation & confusion matrix ===
all_labels = []
all_preds  = []

net.eval()
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, preds = outputs.max(1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('CIFAR-10 Confusion Matrix')
plt.show()


Using device: cuda


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
